In [1]:
import numpy as np
import time

### Load Data

In [2]:
# Hàm để load dữ liệu từ tệp
def load_data(x_path, y_path):
    # Load tín hiệu (x) từ file
    x_data = np.loadtxt(x_path) 
    # Load nhãn (y) từ file
    y_data = np.loadtxt(y_path, dtype=int) 
    return x_data, y_data

# Đường dẫn đến dữ liệu
train_x_path = r"C:\Users\Admin\OneDrive - Hanoi University of Science and Technology\Tailieutruong_20241\AI\HAR\UCI HAR Dataset\UCI HAR Dataset\train\X_train.txt"
train_y_path = r"C:\Users\Admin\OneDrive - Hanoi University of Science and Technology\Tailieutruong_20241\AI\HAR\UCI HAR Dataset\UCI HAR Dataset\train\y_train.txt"
test_x_path = r"C:\Users\Admin\OneDrive - Hanoi University of Science and Technology\Tailieutruong_20241\AI\HAR\UCI HAR Dataset\UCI HAR Dataset\test\X_test.txt"
test_y_path = r"C:\Users\Admin\OneDrive - Hanoi University of Science and Technology\Tailieutruong_20241\AI\HAR\UCI HAR Dataset\UCI HAR Dataset\test\y_test.txt"

# Load dữ liệu
x_train, y_train = load_data(train_x_path, train_y_path)
x_test, y_test = load_data(test_x_path, test_y_path)

# Kiểm tra kích thước
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")


x_train shape: (7352, 561)
y_train shape: (7352,)
x_test shape: (2947, 561)
y_test shape: (2947,)


In [3]:
print(x_train)
print(y_train)

[[ 0.28858451 -0.02029417 -0.13290514 ... -0.84124676  0.17994061
  -0.05862692]
 [ 0.27841883 -0.01641057 -0.12352019 ... -0.8447876   0.18028889
  -0.05431672]
 [ 0.27965306 -0.01946716 -0.11346169 ... -0.84893347  0.18063731
  -0.04911782]
 ...
 [ 0.27338737 -0.01701062 -0.04502183 ... -0.77913261  0.24914484
   0.04081119]
 [ 0.28965416 -0.01884304 -0.15828059 ... -0.78518142  0.24643223
   0.02533948]
 [ 0.35150347 -0.01242312 -0.20386717 ... -0.78326693  0.24680852
   0.03669484]]
[5 5 5 ... 2 2 2]


In [4]:
def one_hot_encode(y, num_classes):
    one_hot = np.zeros((y.size, num_classes))
    one_hot[np.arange(y.size), y - 1] = 1  # Adjusting class index to 0-based
    return one_hot

y_train = one_hot_encode(y_train, num_classes=6)
y_test = one_hot_encode(y_test, num_classes=6)

### Building Neural Network

In [5]:
class NeuralNetwork:
    def __init__(self, sizes, epochs, lr):
        self.sizes = sizes
        self.epochs = epochs
        self.lr = lr

        # Initialize weights with Xavier initialization
        self.params = {
            'W1': np.random.randn(sizes[1], sizes[0]) * np.sqrt(1. / sizes[0]),
            'b1': np.zeros((sizes[1], 1)),
            'W2': np.random.randn(sizes[2], sizes[1]) * np.sqrt(1. / sizes[1]),
            'b2': np.zeros((sizes[2], 1)),
            'W3': np.random.randn(sizes[3], sizes[2]) * np.sqrt(1. / sizes[2]),
            'b3': np.zeros((sizes[3], 1))
        }

    def sigmoid(self, x, derivative=False):
        if derivative:
            return x * (1 - x)  # Efficient derivative
        return 1 / (1 + np.exp(-x))

    def softmax(self, x):
        exps = np.exp(x - np.max(x, axis=0, keepdims=True))  # Stability trick
        return exps / np.sum(exps, axis=0, keepdims=True)

    def forward_pass(self, x):
        self.params['A0'] = x
        self.params['Z1'] = np.dot(self.params['W1'], x) + self.params['b1']
        self.params['A1'] = self.sigmoid(self.params['Z1'])
        self.params['Z2'] = np.dot(self.params['W2'], self.params['A1']) + self.params['b2']
        self.params['A2'] = self.sigmoid(self.params['Z2'])
        self.params['Z3'] = np.dot(self.params['W3'], self.params['A2']) + self.params['b3']
        self.params['A3'] = self.softmax(self.params['Z3'])
        return self.params['A3']

    def backward_pass(self, y, output):
        grads = {}
        error = output - y

        grads['W3'] = np.dot(error, self.params['A2'].T)
        grads['b3'] = np.sum(error, axis=1, keepdims=True)
        error = np.dot(self.params['W3'].T, error) * self.sigmoid(self.params['A2'], derivative=True)

        grads['W2'] = np.dot(error, self.params['A1'].T)
        grads['b2'] = np.sum(error, axis=1, keepdims=True)
        error = np.dot(self.params['W2'].T, error) * self.sigmoid(self.params['A1'], derivative=True)

        grads['W1'] = np.dot(error, self.params['A0'].T)
        grads['b1'] = np.sum(error, axis=1, keepdims=True)
        return grads

    def update_parameters(self, grads):
        for key in self.params.keys():
            if key in grads:
                self.params[key] -= self.lr * grads[key]

    def compute_loss(self, y, output):
        loss = -np.mean(y * np.log(output + 1e-9))  # Cross-entropy
        return loss

    def train(self, x_train, y_train, x_test, y_test):
        for epoch in range(self.epochs):
            start_time = time.time()
            loss = 0
            for i in range(x_train.shape[0]):
                x = x_train[i].reshape(-1, 1)
                y = y_train[i].reshape(-1, 1)
                output = self.forward_pass(x)
                loss += self.compute_loss(y, output)
                grads = self.backward_pass(y, output)
                self.update_parameters(grads)

            loss /= x_train.shape[0]
            accuracy = self.compute_accuracy(x_test, y_test)
            print(f"Epoch {epoch + 1}/{self.epochs}, Loss: {loss:.4f}, Accuracy: {accuracy:.2f}%, Time: {time.time() - start_time:.2f}s")

    def compute_accuracy(self, x_test, y_test):
        correct_predictions = 0
        for i in range(x_test.shape[0]):
            x = x_test[i].reshape(-1, 1)
            y = np.argmax(y_test[i])
            output = self.forward_pass(x)
            if np.argmax(output) == y:
                correct_predictions += 1
        return correct_predictions / x_test.shape[0] * 100


### Training the model

In [7]:
model = NeuralNetwork(sizes=[561, 128, 64, 6], epochs=30, lr=0.01)
model.train(x_train, y_train, x_test, y_test)

Epoch 1/30, Loss: 0.1403, Accuracy: 45.78%, Time: 2.96s
Epoch 2/30, Loss: 0.0602, Accuracy: 74.48%, Time: 2.98s
Epoch 3/30, Loss: 0.0305, Accuracy: 75.33%, Time: 2.86s
Epoch 4/30, Loss: 0.0173, Accuracy: 79.13%, Time: 2.97s
Epoch 5/30, Loss: 0.0115, Accuracy: 83.37%, Time: 4.95s
Epoch 6/30, Loss: 0.0089, Accuracy: 86.56%, Time: 5.18s
Epoch 7/30, Loss: 0.0073, Accuracy: 88.36%, Time: 6.24s
Epoch 8/30, Loss: 0.0063, Accuracy: 90.06%, Time: 5.11s
Epoch 9/30, Loss: 0.0057, Accuracy: 91.01%, Time: 5.11s
Epoch 10/30, Loss: 0.0052, Accuracy: 91.92%, Time: 5.65s
Epoch 11/30, Loss: 0.0049, Accuracy: 92.40%, Time: 6.07s
Epoch 12/30, Loss: 0.0046, Accuracy: 92.91%, Time: 6.41s
Epoch 13/30, Loss: 0.0044, Accuracy: 93.11%, Time: 6.79s
Epoch 14/30, Loss: 0.0042, Accuracy: 93.18%, Time: 6.30s
Epoch 15/30, Loss: 0.0041, Accuracy: 93.32%, Time: 6.62s
Epoch 16/30, Loss: 0.0039, Accuracy: 93.38%, Time: 6.32s
Epoch 17/30, Loss: 0.0038, Accuracy: 93.28%, Time: 6.68s
Epoch 18/30, Loss: 0.0037, Accuracy: 93.

In [11]:
# Lấy trọng số và bias
weights_layer1 = model.params['W1']
bias_layer1 = model.params['b1']

weights_layer2 = model.params['W2']
bias_layer2 = model.params['b2']

weights_layer3 = model.params['W3']
bias_layer3 = model.params['b3']

# In kích thước
print("Shape of W1:", weights_layer1.shape)
print("Shape of b1:", bias_layer1.shape)

print("Shape of W2:", weights_layer2.shape)
print("Shape of b2:", bias_layer2.shape)

print("Shape of W3:", weights_layer3.shape)
print("Shape of b3:", bias_layer3.shape)


Shape of W1: (128, 561)
Shape of b1: (128, 1)
Shape of W2: (64, 128)
Shape of b2: (64, 1)
Shape of W3: (6, 64)
Shape of b3: (6, 1)


In [12]:
# Lưu trọng số và bias 
np.savetxt("weights_layer1.txt", weights_layer1)
np.savetxt("bias_layer1.txt", bias_layer1)

np.savetxt("weights_layer2.txt", weights_layer2)
np.savetxt("bias_layer2.txt", bias_layer2)

np.savetxt("weights_layer3.txt", weights_layer3)
np.savetxt("bias_layer3.txt", bias_layer3)
